In [2]:
using BeamLib
using LinearAlgebra
using StatsBase

# first we set up a test case
# with uncorrelated signals in noise
# arriving at an array

fc = 3.75e9
# Spacing:  4cm is λ/2 for 3.75GHz 
#        subarray 1:  x    x    x   -
#        subarray 2:  -    x    x   x
pa = IsotropicArray(Vector(-1.5:1.5)*4e-2)
Δ = [4e-2, 0, 0]
M = length(pa)
θaxis = Vector(0:1:180)
θgrid = Vector(0:1:180)
Agrid = steer(pa, fc, deg2rad.(θgrid)')

# SNR in dB
function snr2σ²(SNR) 
    return 10^-(SNR/10)
end

Θ = (sort([30, 60]))
d = length(Θ)
A = steer(pa, fc, deg2rad.(Θ)')
Rss = Matrix(I, d, d)
SNR = 10
σ² = snr2σ²(SNR)

N = 1000
s = unconditional_signals(Rss, N)
n = sqrt(σ²/2)*(randn(length(pa), N) + 1im*randn(M, N))
X = A*s + n
Rxx = X*X'/N

# now define a power_func with the estimator we want to sweep with...
pfunc(θ) = music(pa, Rxx, d, fc, θ)[1]

# ... and perform the spectral search 
# note that less DoAs then specified might be returned if not all sources can be resolved
# or spurious peaks might be returned
rad2deg.(vec(find_doas(pfunc, d, range(0, π, 20))))

2-element Vector{Float64}:
 30.34662741692817
 60.09151078706454